In [11]:
import awkward as ak
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
import json
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt

In [12]:
filename = "/home/prayag/Coffea/data/3BF5BC57-5D24-1C4D-B277-7B0BAA5DF3C7.root"
events = NanoEventsFactory.from_root(
    filename,
    schemaclass=NanoAODSchema.v7
).events()

In [17]:
from coffea import util, processor

In [18]:
y = util.load("BackgroundDijets.coffea")
nevents = 0
dict = y["MET_Run2018"]
cutflow_list = []
for key in dict.keys() :
    nevents += dict[key]["Cutflow"]["Total events"]
    cutflow_list.append(dict[key]["Cutflow"])
print("Total events: ", nevents*1e-6 )
print(processor.accumulate(cutflow_list))

Total events:  272.364487
{'Total events': 272364487, 'lumimask': 253027168, 'MET trigger': 138424040, 'MET filters': 130865483, 'MET > 200 GeV': 9283766, 'no electrons': 9283766, 'no muons': 8646681, 'no photons': 8172708, 'no taus': 8172708, 'jet pt > 30': 462191, 'jet abs(eta) < 2.5': 252802, 'deltaphi(jets, MET) > 0.4': 252802, 'At least two bjets': 4247, 'bjet1 pt > 50 GeV and bjet2 pt > 30': 4247, 'Additional Jets <= 2': 2393, 'dijet mass between 100 Gev to 150 GeV': 295, 'dijet pt > 100 GeV': 295}


In [19]:
nevents = 0
dict = y["ZJets_NuNu"]
cutflow_list = []
for key in dict.keys() :
    nevents += dict[key]["Cutflow"]["Total events"]
    cutflow_list.append(dict[key]["Cutflow"])
print("Total events: ", nevents*1e-6 )
print(processor.accumulate(cutflow_list))

Total events:  114.57834199999999
{'Total events': 114578342, 'MET > 200 GeV': 56329228, 'no electrons': 56329228, 'no muons': 56220023, 'no photons': 55862652, 'no taus': 55862652, 'jet pt > 30': 6027300, 'jet abs(eta) < 2.5': 4945716, 'deltaphi(jets, MET) > 0.4': 4945716, 'At least two bjets': 18982, 'bjet1 pt > 50 GeV and bjet2 pt > 30': 18982, 'Additional Jets <= 2': 16959, 'dijet mass between 100 Gev to 150 GeV': 1904, 'dijet pt > 100 GeV': 1903}


In [16]:
util.load("Zjetsnunu_MET_Run2018_from_1_to_20.coffea")

{'MET_Run2018': {'MET_Run2018A': {'Cutflow': {'Total events': 22985157,
    'lumimask': 20020999,
    'MET trigger': 8855754,
    'MET filters': 8143238,
    'MET > 200 GeV': 916265,
    'no electrons': 916265,
    'no muons': 851722,
    'no photons': 805851,
    'no taus': 805851,
    'jet pt > 30': 39327,
    'jet abs(eta) < 2.5': 20328,
    'deltaphi(jets, MET) > 0.4': 20328,
    'At least two bjets': 376,
    'bjet1 pt > 50 GeV and bjet2 pt > 30': 376,
    'Additional Jets <= 2': 208,
    'dijet mass between 100 Gev to 150 GeV': 24,
    'dijet pt > 100 GeV': 24},
   'Histograms': {'met_pt_hist': Hist(Regular(50, 0, 1000, label='Axis 0'), storage=Double()) # Sum: 208.0,
    'met_phi_hist': Hist(Regular(30, -3.14, 3.14, label='Axis 0'), storage=Double()) # Sum: 208.0,
    'leadingjets_pt_hist': Hist(Regular(25, 0, 500, label='Axis 0'), storage=Double()) # Sum: 178.0 (208.0 with flow),
    'leadingjets_eta_hist': Hist(Regular(30, -3, 3, label='Axis 0'), storage=Double()) # Sum: 208.0